## Exercise 63 - 

* Textual file : id\tlongitude\tlatitude\tname,
* Stream of readings : stationId,#free slots,#used slots,#timestamp 

For each reading with a number of free slots equal to 0, print on the standard output timestamp and name of the station. Emit new results every 2 seconds by considering only the data received in the last 2 seconds.

In [1]:
from pyspark.streaming import StreamingContext

In [ ]:
# read the static file containing the stations
inputFileStations = "..."
stationsRDD = sc.textFile(inputFileStations)

# we use cache because the join operation will be done for each batch (each pair as you'll see)
nameStationsRDD = stationsRDD.map(lambda line : (line.split("\t")[0], line.split("\t")[3])).cache()

# create the spark object that wait for the 
# information every two seconds
ssc = StreamingContext(sc,2)

# the stream will be connected to localhost
# through port 9999
lineDStream = ssc.SocketTextStream("localhost",9999)

In [ ]:
# extract only the readings with #free slots equal to 0
# emit (stationId, timestamp)
filteredReadingsDStream = lineDStream.filter(lambda line : line.split(",")[1] == "0")\
                                     .map(lambda line: (line.split(",")[0], line.split(",")[-1]))

# we need to use transform because we cannot compute the join between an RDD and a DStream
# it will return (stationId,(timestamp,name))
joinedNamesTimestampDStream = filteredReadingsDStream.transform(lambda batchRDD : batchRDD.join(nameStationsRDD))

# extract just the value part
namesTimestampsDStream = joinedNamesTimestampDStream.map(lambda pair : pair[1])
                                               
# print on the standard output
namesTimestampsDStream.pprint()

In [ ]:
# start reading the incoming data
ssc.start()

In [ ]:
# stay alive for at most 90 seconds
ssc.awaitTerminationOrTimeout(90)
# stop only the StreamingContext
# but not the SparkContext
ssc.stop(stopSparkContext=False)